In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from IPython.display import display
import pandas as pd
import h5py
import os
from tqdm import tqdm
# to load data from the test file
from utils.create_datasets import SumDatasets
# To load the data sets here, also we will create some examples to explore torch
data_dir = '../data/'
from utils import config
import pickle
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from train_utils import Train

/home/chenxiang/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## given the data, build a very naive summarization model here
* first, we need to compute all the hidden state of each time step and a final state, contain cell state and hidden state of the final time and feed it into the network and do the decoder agian.

In [2]:
train = Train()
train.train_epoches()

  2%|▏         | 10/487 [00:02<02:14,  3.54it/s]

9 have 1.4410


  4%|▍         | 20/487 [00:05<02:09,  3.61it/s]

19 have 1.4380


  6%|▌         | 30/487 [00:08<02:07,  3.58it/s]

...

469 have 0.3469


 99%|█████████▊| 480/487 [02:14<00:01,  3.56it/s]

479 have 0.3469


100%|██████████| 487/487 [02:16<00:00,  3.56it/s]

epoch 10 : Loss:0.34727925062179565
